# Preliminaries

In [165]:
# Import libraries
import networkx as nx
import os

# Define filepath
filepath = 'Instance Files/Saidman_50_NoNDD_Unit_0.txt'

# Strip filename from filepath
filename = os.path.basename(filepath)


# Read in data

In [166]:
def import_kidney_data(filepath):
    """
    Imports kidney exchange data from a text file and structures it into a dictionary.

    Args:
        filepath (str): The path to the text file containing the kidney exchange data.

    Returns:
        dict: A dictionary containing the following keys:
            - num_pairs (int): The number of pairs (donor-patient pairs) in the instance.
            - num_ndd (int): The number of non-directed donors (NDDs) in the instance.
            - num_arcs (int): The total number of arcs in the instance.
            - pairs (list): A list of dictionaries, each representing a pair or NDD with the following keys:
                - id (int): The unique ID of the pair.
                - is_ndd (bool): True if the pair is an NDD, False otherwise.
                - donor_blood_type (int): Donor's blood type (0 = A, 1 = B, 2 = AB, 3 = O).
                - patient_blood_type (int): Patient's blood type (0 = A, 1 = B, 2 = AB, 3 = O).
                - patient_vpra (int): Patient's vPRA score (0 = below 0.5, 1 = between 0.5 and 0.85, 2 = above 0.85).
            - arcs (list): A list of dictionaries representing the arcs between pairs with the following keys:
                - donor_id (int): ID of the donor pair.
                - patient_id (int): ID of the patient pair.
                - weight (int): The weight of the arc (always 1 in this case).

    Example:
        data = import_kidney_data('path/to/data.txt')
        print(data['num_pairs'])  # Output: Number of pairs in the instance
    """

    data = {}
    
    # Open the file and read all lines into memory
    with open(filepath, 'r') as f:
        lines = f.readlines()
        
        # Parse metadata: number of pairs, NDDs, and arcs
        num_pairs = int(lines[0].split(' ')[2])
        num_ndd = int(lines[1].split(' ')[2])
        num_arcs = int(lines[2].split(' ')[2])
        
        # Store the parsed values in the data dictionary
        data['num_pairs'] = num_pairs
        data['num_ndd'] = num_ndd
        data['num_arcs'] = num_arcs
        
        # Total number of entities (pairs + NDDs)
        num_things = num_pairs + num_ndd
        
        # List to store information about pairs and NDDs
        pairs = []
        
        # Parse each line corresponding to pairs and NDDs
        for line in lines[3:num_things + 3]:
            id, is_ndd, donor_blood_type, patient_blood_type, patient_vpra = map(int, line.strip().split(','))
            
            # Add pair/NDD info to the list
            pairs.append({
                'id': id,
                'is_ndd': bool(is_ndd),
                'donor_blood_type': donor_blood_type,
                'patient_blood_type': patient_blood_type,
                'patient_vpra': patient_vpra,
            })
        
        # Store the pairs data in the dictionary
        data['pairs'] = pairs

        # List to store information about arcs
        arcs = []
        
        # Parse each line corresponding to arcs (donor-patient relationships)
        for line in lines[num_things + 3:]:
            arc, weight = line.strip().split(',1,')
            
            # Extract donor and patient IDs from the arc
            donor_id, patient_id = arc.split(',')
            weight = int(weight.strip())  # Weight is always 1 in this context
            donor_id = int(donor_id[1:])  # Remove prefix and convert to int
            patient_id = int(patient_id[:-1])  # Remove suffix and convert to int
            
            # Add arc info to the list
            arcs.append({
                'donor_id': donor_id,
                'patient_id': patient_id,
                'weight': weight,
            })
        
        # Store the arcs data in the dictionary
        data['arcs'] = arcs

    # Return the final structured dictionary
    return data


# Example usage of the function
data = import_kidney_data(filepath)


In [167]:
# Display the number of pairs, NDDs, and arcs
print(f"Kidney exchange data from: {filename}")
print(f"Number of pairs: {data['num_pairs']}")
print(f"Number of NDDs: {data['num_ndd']}")
print(f"Number of arcs: {data['num_arcs']}")
print('')

# Access information about a specific arc
arc_id = 205
arc = data['arcs'][arc_id] 
print(f"Arc ID: {arc_id}")
print(f"Donor ID: {arc['donor_id']}")
print(f"Patient ID: {arc['patient_id']}")
print(f"Weight: {arc['weight']}")
print('')

# Access information about a specific pair (based on pair ID)
pair_id = 49
pair = data['pairs'][pair_id]
print(f"Pair ID: {pair_id}")
print(f"Is NDD: {pair['is_ndd']}")
print(f"Donor blood type: {pair['donor_blood_type']}")
print(f"Patient blood type: {pair['patient_blood_type']}")
print(f"Patient vPRA: {pair['patient_vpra']}")
print('')

# Print the complete data for debugging purposes (optional)
#print(data)


Kidney exchange data from: Saidman_50_NoNDD_Unit_0.txt
Number of pairs: 50
Number of NDDs: 0
Number of arcs: 472

Arc ID: 205
Donor ID: 19
Patient ID: 21
Weight: 1

Pair ID: 49
Is NDD: False
Donor blood type: 0
Patient blood type: 0
Patient vPRA: 0



# Define graph functions

In [199]:
import numpy as np

def create_graph(data):
    """
    Creates a directed graph from the input data.

    Parameters:
    data (dict): A dictionary with two keys:
        - 'pairs': A list of dictionaries, each containing an 'id' for a node.
        - 'arcs': A list of dictionaries, each containing 'donor_id' (start node), 
                  'patient_id' (end node), and 'weight' (weight of the edge).

    Returns:
    G (nx.DiGraph): A directed graph created from the input data.
    """
    G = nx.DiGraph()
    
    # Add nodes to the graph from 'pairs' in data
    for pair in data['pairs']:
        G.add_node(pair['id'])
    
    # Add edges (directed) with weights from 'arcs' in data
    for arc in data['arcs']:
        G.add_edge(arc['donor_id'], arc['patient_id'], weight=arc['weight'])
    
    return G

In [200]:
def findPath(G, u, n):
    """
    Recursively finds all paths of length n starting from node u.

    Parameters:
    G (nx.DiGraph): A directed graph.
    u (hashable): The starting node of the path.
    n (int): The length of the paths to find.

    Returns:
    paths (list): A list of all possible paths of length n starting from node u. 
                  Each path is represented as a list of nodes.
    """
    if n == 0:
        # Base case: if the path length is 0, return the starting node as the path
        return [[u]]
    
    # Recursive case: explore neighbors of the current node and find paths of length n-1
    paths = [[u] + path for neighbor in G.successors(u) for path in findPath(G, neighbor, n-1)]
    
    return paths



def find_cycle(G, u, n):
    """
    Finds all cycles of length n starting and ending at node u.

    Parameters:
    G (nx.DiGraph): A directed graph.
    u (hashable): The node where the cycle starts and ends.
    n (int): The length of the cycle (number of edges in the cycle).

    Returns:
    cycles (list): A list of all cycles of length n starting and ending at node u.
                   Each cycle is represented as a tuple of nodes.
    """
    # Find all paths of length n starting from node u
    paths = findPath(G, u, n)
    
    # Filter paths that form a cycle, meaning they end at u and visit u exactly twice
    return [tuple(path) for path in paths if (path[-1] == u) and sum(x == u for x in path) == 2]

In [201]:
def set_weight(dict_list, donor_id, patient_id, weight_key="weight"):
    """
    Retrieve the weight of an arc between a donor and a patient.

    Args:
        dict_list (list): List of dictionaries representing arcs.
        donor_id (int): The ID of the donor.
        patient_id (int): The ID of the patient.
        weight_key (str): The key to access the weight in the dictionary (default is "weight").

    Returns:
        int: The weight of the arc if found, otherwise None.
    """
    for dct in dict_list:
        if dct.get("donor_id") == donor_id and dct.get("patient_id") == patient_id:
            return dct.get(weight_key)
    return None  # Return None if no matching arc is found


#Faster way to find weight of an arc using adjacency matrix
def get_weight(mat, donor_id, patient_id):
    return int(mat[donor_id][patient_id])

In [202]:
cycle_times = []

In [203]:
import time

start_graph = time.time()

G = create_graph(data) # Create a directed graph from the data

end_graph = time.time()

#create weight matrix for faster lookup
num_things = len(G.nodes)
weight_matrix = np.zeros((num_things, num_things))
print('Graph time: {}'.format(end_graph-start_graph))


for arc in data['arcs']:
    weight_matrix[arc['donor_id']][arc['patient_id']] = arc['weight']

k = 3  # Maximum length for cycles and paths
 # List to store information about cycles
p = []  # List to store information about paths
for i in range(20):
    c = [] 
    startc = time.time()
    id_count = 0
    # Loop through possible cycle lengths from 1 to k
    for l in range(2, k+1):
        for node in G.nodes:  # Iterate over all nodes in the graph
            for cyc in list(find_cycle(G, node, l)):
                # Find all cycles starting at this node of length 'l'
                cyc_success = 0
                cyc_weight = 0
                
                # Calculate the success and weight for the cycle
                for n in range(1, len(cyc)):
                    cyc_success += data['pairs'][cyc[n]]['patient_vpra']  # Sum up the vPRA values of patients in the cycle
                    cyc_weight += set_weight(data['arcs'], cyc[n-1], cyc[n])  # Add the weight of the arc between consecutive pairs in the cycle
                    #cyc_weight += get_weight(weight_matrix, cyc[n-1], cyc[n])  # Add the weight of the arc using adjacency matrix
                
                # Append the cycle info to the list 'c'
                c.append({'id': id_count,'cycle': cyc, 'vpra_sum': cyc_success, 'weight_sum': cyc_weight})  
                id_count += 1
    
    endc = time.time()
    
    print('cycle time: {}'.format(endc-startc))
    cycle_times.append(endc-startc)
    startp = time.time()
    # Loop through possible path lengths from 1 to k-2 (since paths are from NDDs)
    for l in range(1, k):
        for node in G.nodes:  # Iterate over all nodes in the graph
            if data['pairs'][node]['is_ndd']:  # Only consider nodes that are NDDs
                for path in list(findPath(G, node, l)):  # Find all paths starting at this NDD node of length 'l'
                    path_success = 0
                    path_weight = 0
                    
                    # Calculate the success and weight for the path
                    for n in range(1, len(path)):
                        path_success += data['pairs'][path[n]]['patient_vpra']  # Sum up the vPRA values of patients in the path
                        path_weight += set_weight(data['arcs'], cyc[n-1], cyc[n])  # Add the weight of the arc between consecutive pairs in the path
                    # Append the path info to the list 'p'
                    
                    c.append({'id':id_count,'cycle': path, 'vpra_sum': path_success, 'weight_sum': path_weight}) 
                    id_count += 1
    endp = time.time()
    
    print('path time: {}'.format(endp-startp))
# Print the results
print("Cycles/paths:", c) 
#print("Paths:", p)

Graph time: 0.02878260612487793
cycle time: 5.308985710144043
path time: 0.00010442733764648438
cycle time: 0.8967628479003906
path time: 6.532669067382812e-05
cycle time: 1.0085768699645996
path time: 6.461143493652344e-05
cycle time: 0.7183041572570801
path time: 0.00012969970703125
cycle time: 0.1758866310119629
path time: 7.104873657226562e-05
cycle time: 0.08584308624267578
path time: 9.393692016601562e-05
cycle time: 1.7836496829986572
path time: 7.462501525878906e-05
cycle time: 0.3756873607635498
path time: 6.556510925292969e-05
cycle time: 0.9092254638671875
path time: 0.00031280517578125
cycle time: 0.5735905170440674
path time: 0.000110626220703125
cycle time: 0.7995297908782959
path time: 0.00010037422180175781
cycle time: 0.615703821182251
path time: 8.559226989746094e-05
cycle time: 0.3722407817840576
path time: 8.559226989746094e-05
cycle time: 2.397231101989746
path time: 0.0003840923309326172
cycle time: 3.802643299102783
path time: 0.0001990795135498047
cycle time: 1.

## NEW VERSION of cycle generatio below

### old one is kept above for testing. Adjust for loop if you only want to run once

In [204]:
import networkx as nx

def canonical_form(cycle):
    """
    Returns the canonical form of a cycle by rotating it to start with the smallest node.
    """
    min_index = cycle.index(min(cycle))
    return tuple(cycle[min_index:] + cycle[:min_index])

def findPaths(G, u, n, memo):
    """
    Recursively finds all paths of length n starting from node u, using memoization.
    
    Parameters:
    G (nx.DiGraph): A directed graph.
    u (hashable): The starting node of the path.
    n (int): The length of the paths to find.
    memo (dict): A memoization dictionary for storing already computed paths.

    Returns:
    paths (list): A list of all possible paths of length n starting from node u.
    """
    if (u, n) in memo:
        return memo[(u, n)]

    if n == 0:
        return [[u]]

    paths = [[u] + path for neighbor in G.successors(u) for path in findPaths(G, neighbor, n-1, memo)]
    
    memo[(u, n)] = paths
    return paths

def is_valid_cycle(path):
    """
    Checks if a path forms a valid cycle.
    
    A valid cycle must start and end at the same node, and no internal node should repeat.
    """
    return path[-1] == path[0] and len(set(path[:-1])) == len(path) - 1

def find_cycles(G, u, n, memo):
    """
    Finds all cycles of length n starting and ending at node u, and ensures no duplicates using canonical form.

    Parameters:
    G (nx.DiGraph): A directed graph.
    u (hashable): The node where the cycle starts and ends.
    n (int): The length of the cycle (number of edges in the cycle).
    memo (dict): A memoization dictionary for storing already computed paths.

    Returns:
    cycles (list): A list of all cycles of length n starting and ending at node u.
    """
    # Find all paths of length n starting from node u
    paths = findPaths(G, u, n, memo)

    # Filter paths that form a valid cycle and return in canonical form
    cycles = [canonical_form(path) for path in paths if is_valid_cycle(path)]

    return list(set(cycles))

def find_all_cycles_in_sccs(G, k):
    """
    Finds all unique cycles of length up to k in the strongly connected components of the graph.
    
    Parameters:
    G (nx.DiGraph): A directed graph.
    k (int): The maximum length of the cycles.

    Returns:
    cycles (list): A list of all unique cycles in the graph, up to length k.
    """
    c = set()  # Use a set to store unique cycles

    # Get all strongly connected components
    sccs = nx.strongly_connected_components(G)

    for scc in sccs:
        subgraph = G.subgraph(scc)  # Only search within the SCC
          # Memoization dictionary for storing already computed paths
        for l in range(2, k + 1):
            memo = {}
            for node in subgraph.nodes:
                for cyc in find_cycles(subgraph, node, l, memo):
                    if is_valid_cycle(cyc):
                        c.add(cyc)  # Store the canonical form

    return list(c)

# Test usage:
test = nx.DiGraph()
test.add_edges_from([(1, 2), (2, 3), (3, 1), (3, 4), (4, 2), (2, 5)])

# Find all unique cycles of length up to 3
k = 3
cycles = find_all_cycles_in_sccs(test, k)
print(cycles)


[(2, 3, 4, 2), (1, 2, 3, 1)]


In [205]:
cycle_time_new = []

In [206]:

import time

start_graph = time.time()

G = create_graph(data) # Create a directed graph from the data

end_graph = time.time()

#create weight matrix for faster lookup
num_things = len(G.nodes)
weight_matrix = np.zeros((num_things, num_things))
print('Graph time: {}'.format(end_graph-start_graph))


for arc in data['arcs']:
    weight_matrix[arc['donor_id']][arc['patient_id']] = arc['weight']

k = 3  # Maximum length for cycles and paths
 # List to store information about cycles
p = []  # List to store information about paths

#### How many times to run (to test runtime)  Make 1 to generate cycles once
for i in range(10):
    c = [] 
    startc = time.time()
    id_count = 0
    for cyc in list(find_all_cycles_in_sccs(G, k)):
        # Find all cycles starting at this node of length 'l'
        cyc_success = 0
        cyc_weight = 0
                
        # Calculate the success and weight for the cycle
        for n in range(1, len(cyc)):
            cyc_success += data['pairs'][cyc[n]]['patient_vpra']  # Sum up the vPRA values of patients in the cycle
            #cyc_weight += set_weight(data['arcs'], cyc[n-1], cyc[n])  # Add the weight of the arc between consecutive pairs in the cycle
            cyc_weight += get_weight(weight_matrix, cyc[n-1], cyc[n])  # Add the weight of the arc between consecutive pairs in the cycle
                
        # Append the cycle info to the list 'c'
        c.append({'id': id_count,'cycle': cyc, 'vpra_sum': cyc_success, 'weight_sum': cyc_weight})  
        id_count += 1
    
    endc = time.time()
    
    print('cycle time: {}'.format(endc-startc))
    cycle_time_new.append(endc-startc)
    startp = time.time()
    # Loop through possible path lengths from 1 to k-2 (since paths are from NDDs)
    for l in range(1, k):
        for node in G.nodes:  # Iterate over all nodes in the graph
            if data['pairs'][node]['is_ndd']:  # Only consider nodes that are NDDs
                for path in list(findPath(G, node, l)):  # Find all paths starting at this NDD node of length 'l'
                    path_success = 0
                    path_weight = 0
                
                    # Calculate the success and weight for the path
                    for n in range(1, len(path)):
                        path_success += data['pairs'][path[n]]['patient_vpra']  # Sum up the vPRA values of patients in the path
                        path_weight += get_weight(weight_matrix, path[n-1], path[n])  # Add the weight of the arc between consecutive pairs in the path
                    # Append the path info to the list 'p'
                    
                    c.append({'id':id_count,'cycle': path, 'vpra_sum': path_success, 'weight_sum': path_weight}) 
                    id_count += 1
    endp = time.time()
    
    print('path time: {}'.format(endp-startp))
# Print the results
print("Cycles/paths:", c) 
#print("Paths:", p)

Graph time: 0.14050555229187012
cycle time: 0.08100533485412598
path time: 0.0001304149627685547
cycle time: 0.0890805721282959
path time: 0.000209808349609375
cycle time: 0.16534662246704102
path time: 0.00011372566223144531
cycle time: 0.08874917030334473
path time: 0.0004432201385498047
cycle time: 0.03227496147155762
path time: 8.702278137207031e-05
cycle time: 0.022499799728393555
path time: 7.295608520507812e-05
cycle time: 0.03415036201477051
path time: 7.867813110351562e-05
cycle time: 0.018719911575317383
path time: 0.0001418590545654297
cycle time: 0.018579721450805664
path time: 7.867813110351562e-05
cycle time: 0.040818214416503906
path time: 0.00012826919555664062
Cycles/paths: [{'id': 0, 'cycle': (8, 42, 49, 8), 'vpra_sum': 0, 'weight_sum': 3}, {'id': 1, 'cycle': (8, 42, 8), 'vpra_sum': 0, 'weight_sum': 2}, {'id': 2, 'cycle': (23, 37, 25, 23), 'vpra_sum': 2, 'weight_sum': 3}, {'id': 3, 'cycle': (16, 29, 22, 16), 'vpra_sum': 0, 'weight_sum': 3}, {'id': 4, 'cycle': (20, 29,

In [207]:
import statistics


print('old method avg time', statistics.mean(cycle_times))
print('new method avg time:', statistics.mean(cycle_time_new))

old method avg time 1.314550244808197
new method avg time: 0.059122467041015626


In [57]:
def largest_weight_heuristic(cycles):
    """
    Implements the largest weight heuristic for selecting cycles in the kidney exchange problem.

    Args:
        cycles: A list of cycles, where each cycle is represented as a dictionary with keys 'id', 'cyc', and 'weight'.

    Returns:
        A list of selected cycles.
    """

    selected_cycles = []
    while cycles:
        # Find the cycle with the largest weight
        max_weight_cycle = max(cycles, key=lambda c: c['weight_sum'])

        # Remove cycles that are not disjoint from the selected cycles
        disjoint_cycles = [c for c in cycles if not set(max_weight_cycle['cycle']).intersection(set(c['cycle']))]

        # Add the selected cycle to the list of selected cycles
        selected_cycles.append(max_weight_cycle)
        cycles = disjoint_cycles
    transplants_made = 0    
    for c in selected_cycles:
        transplants_made += len(c['cycle'])-1
    return selected_cycles, transplants_made

In [188]:
selected_cycles, transplants_made = largest_weight_heuristic(c)
print(selected_cycles)
print('total transplants: {}'.format(transplants_made))

[{'id': 30, 'cycle': (8, 42, 10, 8), 'vpra_sum': 0, 'weight_sum': 3}, {'id': 43, 'cycle': (13, 37, 16, 13), 'vpra_sum': 0, 'weight_sum': 3}, {'id': 49, 'cycle': (20, 29, 22, 20), 'vpra_sum': 0, 'weight_sum': 3}, {'id': 14, 'cycle': (23, 25, 23), 'vpra_sum': 2, 'weight_sum': 2}, {'id': 26, 'cycle': (45, 49, 45), 'vpra_sum': 0, 'weight_sum': 2}]
total transplants: 13


In [ ]:
def is_valid_cycle(cycle):
    """
    Checks if a given cycle is valid in a kidney exchange problem.

    Args:
        cycle: A list of pairs (donor_id, recipient_id) representing the cycle.
        compatibility_graph: A graph representing compatibility between donors and recipients.
        used_donors: A set of used donors.
        used_recipients: A set of used recipients.

    Returns:
        True if the cycle is valid, False otherwise.
    """
    arcs = data['arcs']
    for i in range(len(cycle)-1):
        
        if any(edge['donor_id'] == cycle[i] and edge['patient_id'] == cycle[i+1] for edge in arcs):
            continue
        else:
            return False

    return True




def heuristic_with_lns(cycles, unassigned_vertecies, edge_weights):
    """
    Implements the largest weight heuristic for selecting cycles in the kidney exchange problem.

    Args:
        cycles: A list of cycles, where each cycle is represented as a dictionary with keys 'id', 'cyc', and 'weight'.

    Returns:
        A list of selected cycles.
    """

    selected_cycles = []
    while cycles:
        # Find the cycle with the largest weight
        max_weight_cycle = max(cycles, key=lambda c: c['weight_sum'])

        # Remove cycles that are not disjoint from the selected cycles
        disjoint_cycles = [c for c in cycles if not set(max_weight_cycle['cycle']).intersection(set(c['cycle']))]

        # Add the selected cycle to the list of selected cycles
        selected_cycles.append(max_weight_cycle)
        cycles = disjoint_cycles
        
    # Remove used vertices from unassigned_vertices
    for cycle in selected_cycles:
        for vertex in cycle['cycle']:
            if vertex in unassigned_vertecies:
                unassigned_vertecies.remove(vertex)
    
    # Step 4: Try to convert 2-way cycles to 3-way cycles
    for cycle in selected_cycles:
        if len(cycle['cycle']) == 2:
            for unassigned_vertex in unassigned_vertecies:
                # Find the edge weights based on donor_id and patient_id
                weight1 = next((edge['weight'] for edge in edge_weights if edge['donor_id'] == cycle['cycle'][0] and edge['patient_id'] == unassigned_vertex), 0)
                weight2 = next((edge['weight'] for edge in edge_weights if edge['donor_id'] == unassigned_vertex and edge['patient_id'] == cycle['cycle'][1]), 0)

                new_cycle = {'id': cycle['id'], 'cycle': cycle['cycle'] + [unassigned_vertex], 'weight_sum': cycle['weight_sum'] + weight1 + weight2}
                if is_valid_cycle(new_cycle['cycle']):
                    selected_cycles.remove(cycle)
                    selected_cycles.append(new_cycle)
                    unassigned_vertices.remove(unassigned_vertex)
                    break

    # Step 5: Try to convert 3-way cycles to two disjoint 2-way cycles
    for cycle in selected_cycles:
        if len(cycle['cycle']) == 3:
            for unassigned_vertex in unassigned_vertecies:
                # Find the edge weights based on donor_id and patient_id
                weight1 = next((edge['weight'] for edge in edge_weights if edge['donor_id'] == cycle['cycle'][0] and edge['patient_id'] == unassigned_vertex), 0)
                weight2 = next((edge['weight'] for edge in edge_weights if edge['donor_id'] == unassigned_vertex and edge['patient_id'] == cycle['cycle'][1]), 0)
                weight3 = next((edge['weight'] for edge in edge_weights if edge['donor_id'] == cycle['cycle'][1] and edge['patient_id'] == cycle['cycle'][2]), 0)

                cycle1 = {'id': cycle['id'] + 1, 'cycle': [cycle['cycle'][0], unassigned_vertex], 'weight_sum': weight1 + weight2}
                cycle2 = {'id': cycle['id'] + 2, 'cycle': [unassigned_vertex, cycle['cycle'][1], cycle['cycle'][2]], 'weight_sum': weight2 + weight3}
                if is_valid_cycle(cycle1['cycle']) and is_valid_cycle(cycle2['cycle']):
                    selected_cycles.remove(cycle)
                    selected_cycles.append(cycle1)
                    selected_cycles.append(cycle2)
                    unassigned_vertices.remove(unassigned_vertex)
                    break

    return selected_cycles

In [ ]:
uv = data['pairs']
ew = data['arcs']
selected_cycles = heuristic_with_lns(c, uv, ew)
print(selected_cycles)

transplants_made = 0
for c in selected_cycles:
    transplants_made += len(c['cycle'])-1
print(transplants_made)